<a href="https://colab.research.google.com/github/Patrick-Sinnott/Quantum-Algorithms---Assignments/blob/main/QA_Assignment_3_(Includes_VQE)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem set 3

## General directions
<ul>
    <li>Some of the exercises in the following notebook are numerical, and should be performed directly within the notebook. Other exercises are analytical, and should be solved on attached documents. In the latter case, handwritten papers are acceptable, but please make sure to write in a comprehensible manner.</li>
    <li>For the numerical tasks, please comment your code to explain what does what. Use meaningful names for variables and functions. </li>
    <li>We will need to be able to run the notebook. Make sure that there are no dependencies in the notebook based on files on your computer!</li>
    <li>Feel free to look online for help! Python documentation is <a href="https://docs.python.org/3/">here</a>, NumPy documentation <a href="https://numpy.org/doc/stable/">here</a> and Qiskit manual is <a href="https://qiskit.org/documentation/">here</a>. Some specific pages of Qiskit documentation or other useful sources are linked in the relevant questions.</li>
</ul>


$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

# 3.0 - Dependencies
Please add all the relevant dependencies for the problems to the following cell and avoid the <code>import</code> command elsewhere.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import qiskit

from qiskit import *
import numpy as np
from numpy import linalg as la
from qiskit.tools.monitor import job_monitor
import qiskit.tools.jupyter
from qiskit.circuit import QuantumCircuit, Parameter
import scipy
from math import log2 as log2
aer_sim = Aer.get_backend('aer_simulator')

zero = np.array([1,0])
one = np.array([0,1])

sx = np.array([[0, 1],  [ 1, 0]], dtype=np.complex128)
sy = np.array([[0, -1j],[1j, 0]], dtype=np.complex128)
sz = np.array([[1, 0],  [0, -1]], dtype=np.complex128)
id = np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
S = [id, sx, sy, sz]


# 3.1 - Quantum Fourier Transform (3 pts.)

Consider the unitary operator $\mathcal{F}$ for the Quantum Fourier Transform
$$\mathcal{F} \ket{j} = \frac{1}{\sqrt{N}}\sum_{k=0}^{N-1} e^{2\pi i j k /N}\ket{k}.$$
<ol>
    <li>Compute explicitly the Fourier transform on the $n$ qubits state $\ket{0\ldots 0}$</li>
    <li>We already know a quantum circuit able to perform the Fourier Transform. Give a quantum circuit for the inverse Quantum Fourier Transform (i.e., for the operator $\mathcal{F}^\dagger$) on 5 qubits. Create that circuit in Qiskit. Transform the circuit into a gate using the adequate functions.
        <li>In Qiskit, create the circuit for the Fourier Transform $\mathcal{F}$ on 5 qubits as well, and transform it into a gate.</li>
    <li>Again using Qiskit, create the circuits $\mathcal{F}\mathcal{F}^\dagger$ and $\mathcal{F}^\dagger\mathcal{F}$. Obtain the matrix representation of the two circuits, and show that it is equal to the identity in both cases.</li>
</ol>

In [ ]:
N=5

def inv_qft(qc, n):
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
       

def qft_rotations(circuit, n):
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(np.pi/2**(n-qubit), qubit, n)
    qft_rotations(circuit, n)

def qft(circuit, n):
    qft_rotations(circuit, n)
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit
    
qc = qiskit.QuantumCircuit(N)
inv_qft(qc,N)
qftdag=qc.to_gate()
qc.draw()

┌───┐                                                            »
q_0: ─X─┤ H ├─■──────────────■───────────────────■───────────────────────»
      │ └───┘ │P(-π/2) ┌───┐ │                   │                       »
q_1: ─┼───X───■────────┤ H ├─┼─────────■─────────┼──────────────■────────»
      │   │            └───┘ │P(-π/4)  │P(-π/2)  │        ┌───┐ │        »
q_2: ─┼───┼──────────────────■─────────■─────────┼────────┤ H ├─┼────────»
      │   │                                      │P(-π/8) └───┘ │P(-π/4) »
q_3: ─┼───X──────────────────────────────────────■──────────────■────────»
      │                                                                  »
q_4: ─X──────────────────────────────────────────────────────────────────»
                                                                         »
«                                                                  
«q_0: ─■───────────────────────────────────────────────────────────
«      │                                                           
«q_1: ─┼────────────────────■──────────────────────────────────────
«      │                    │                                      
«q_2: ─┼──────────■─────────┼──────────────■───────────────────────
«      │          │P(-π/2)  │        ┌───┐ │                       
«q_3: ─┼──────────■─────────┼────────┤ H ├─┼─────────■─────────────
«      │P(-π/16)            │P(-π/8) └───┘ │P(-π/4)  │P(-π/2) ┌───┐
«q_4: ─■────────────────────■──────────────■─────────■────────┤ H ├
«                                                             └───┘

In [ ]:
qc = QuantumCircuit(N)
qft(qc,N)
qc.draw()
qft=qc.to_gate(label='QFT')
qc.draw()

»
q_0: ──────■─────────────────────────────────────────■─────────────────────────»
           │                                         │                         »
q_1: ──────┼─────────■───────────────────────────────┼────────■────────────────»
           │         │                               │        │                »
q_2: ──────┼─────────┼────────■──────────────────────┼────────┼────────■───────»
           │         │        │                ┌───┐ │P(π/8)  │P(π/4)  │P(π/2) »
q_3: ──────┼─────────┼────────┼────────■───────┤ H ├─■────────■────────■───────»
     ┌───┐ │P(π/16)  │P(π/8)  │P(π/4)  │P(π/2) └───┘                           »
q_4: ┤ H ├─■─────────■────────■────────■───────────────────────────────────────»
     └───┘                                                                     »
«                                          ┌───┐   
«q_0: ──────■──────────────────────■───────┤ H ├─X─
«           │                ┌───┐ │P(π/2) └───┘ │ 
«q_1: ──────┼────────■───────┤ H ├─■─────────X───┼─
«     ┌───┐ │P(π/4)  │P(π/2) └───┘           │   │ 
«q_2: ┤ H ├─■────────■───────────────────────┼───┼─
«     └───┘                                  │   │ 
«q_3: ───────────────────────────────────────X───┼─
«                                                │ 
«q_4: ───────────────────────────────────────────X─
«

In [ ]:
qubs = np.arange(N)
qubs = list(qubs)
qc1 = QuantumCircuit(N)
qc1.append(qftdag,qubs)
qc1.append(qft,qubs)
backend = Aer.get_backend('unitary_simulator')
job = execute(qc1, backend)
result = job.result()
print(result.get_unitary(qc1, decimals=3))

Operator([[ 1.-0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j, -0.-0.j,  0.+0.j],
          [ 0.+0.j,  1.-0.j,  0.+0.j, ..., -0.-0.j,  0.+0.j, -0.-0.j],
          [-0.-0.j,  0.+0.j,  1.-0.j, ...,  0.+0.j, -0.+0.j,  0.+0.j],
          ...,
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  1.-0.j,  0.+0.j,  0.-0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  1.-0.j,  0.+0.j],
          [ 0.+0.j,  0.+0.j,  0.+0.j, ..., -0.+0.j,  0.+0.j,  1.-0.j]],
         input_dims=(2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2))


In [ ]:
qc2 = QuantumCircuit(N)
qc2.append(qft,qubs)
qc2.append(qftdag,qubs)
backend = Aer.get_backend('unitary_simulator')
job = execute(qc2, backend)
result = job.result()
print(result.get_unitary(qc2, decimals=3))

Operator([[ 1.-0.j, -0.+0.j, -0.-0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
          [ 0.+0.j,  1.-0.j, -0.-0.j, ..., -0.+0.j,  0.+0.j,  0.-0.j],
          [ 0.+0.j,  0.+0.j,  1.-0.j, ..., -0.+0.j,  0.+0.j,  0.+0.j],
          ...,
          [-0.-0.j, -0.-0.j, -0.+0.j, ...,  1.-0.j, -0.+0.j, -0.+0.j],
          [-0.-0.j, -0.+0.j, -0.+0.j, ...,  0.+0.j,  1.-0.j, -0.-0.j],
          [-0.-0.j, -0.-0.j, -0.+0.j, ...,  0.-0.j,  0.+0.j,  1.-0.j]],
         input_dims=(2, 2, 2, 2, 2), output_dims=(2, 2, 2, 2, 2))


We can see that these both matrices are equal to unity

# 3.2 - Quantum Phase Estimation (6 pts. total)

## 3.2.a Qiskit code /1 (3 pts.)

<ol>
    <li>In Qiskit, create the quantum circuit for quantum phase estimation of a <code>T-gate</code>. How many qubits do you need to consider to obtain an exact value for the phase for the eigenstate $\ket{0}$? And for the eigenstate $\ket{1}$?</li>
    <li>Create a complete Python function <code>phase_estimation(oracle, eigenstate, number_qubits)</code> which performs the phase estimation for a generic oracle <code>oracle</code>, on the eigenstate <code>eigenstate</code>, and employing <code>number_qubits</code> qubits. The function should perform a quantum simulation, and return the estimated value of the phase (i.e., to bit-precision of <code>number_qubits</code>). Apply the function to the <code>T-gate</code> of the previous exercise, and verify that it works as expected.</li>
    <li>Now, run the function with a different oracle, given by $R_z(\frac{1}{3})$. What happens?</li>
    <li>In a graph, plot the highest-probability result obtained for the phase estimation on $R_z(\frac{1}{3})$, as a function of the number of employed qubits <code>number_qubits</code>.</li>
</ol>

## 3.2.b Analytical treatment of the error (3 pts.)
After the controlled application of the oracle, but before the final Fourier Transform, the state of the $m$-qubits first register is:
$$\ket{\theta} = \frac{1}{2^m} \sum_{k=0}^{2^m-1} e^{2\pi i k \theta} \ket{k}$$
where $\theta$ is the parameter to be estimated. 

<ol>
    <li>Apply the transform to $\ket{\theta}$, and give and analytical expression for the output state.</li>
    <li>Determine the probability to obtain an outcome $j \in \{0,\ldots,2^m-1\}$ when performing the measurement.</li>
    <li>Show that, if there exists a $j=\tilde{j}$ such that $\theta=\frac{\tilde{j}}{2^m}$ (i.e. $\theta$ has an exact representation in $m$ bits), then the outcome of the measurement is $j$ with unitary probability.</li>
    <li>If this is not the case, let $j^*$ be the specific value of $j$ which gives the best estimation $\frac{j^*}{2^m}$ of $\theta$. In other terms, it should be true that $\theta = \frac{j}{2^m} +\varepsilon$, where $|\varepsilon| < 2^{-(m+1)}$. Prove that the probability of getting the outcome $j^*$ for the measurement is larger than $\frac{4}{\pi^2}$. </li>
</ol>

<b>Hint/1.</b> You could find useful the formula for the summation of geometric progression, see <a href="https://en.wikipedia.org/wiki/Geometric_progression">here</a>. <br>
<b>Hint/2.</b> The following inequalities are given:
$$\frac{2\pi |\varepsilon| 2^m}{|e^{2\pi i \varepsilon 2^m}-1|}\leq \frac{\pi}{2}$$ <br>
$$|e^{2\pi i \varepsilon} - 1| \leq 2\pi |\varepsilon|$$

# 3.3 - Period finding (6 pts. total)
Consider a function $f:\mathbb{Z}_N\to C$, where $C$ is a finite set of values. Suppose that $f$ is known to be periodic; namely:
$$\exists s \in Z_N - \{0\} \mbox{ s.t. } f(x+s)=f(x)\,\,\,\, \forall x \in Z_N$$
Furthermore, in each period, each symbol appears only once.
The goal of this problem is to obtain $s$. For the sake of simplicity, assume $N = 2^n$ for some $n$.

## 3.3.a Classical version (1 pt.)
<ol>
  <li>Show that the periodicity condition implies that $s$ divides $N$. </li>
  <li>Show that the problem can be solved on a classical calculator with $\mathcal{O}(n)$ evaluations of the function $f$, where $n = \log_2 N$.  </li>
</ol>

## 3.3.b Quantum protocol (2 pts.)
Consider an oracle $O_f$, acting as follows:
$$O_f(\ket{x}\ket{b}) = \ket{x}\ket{b\oplus f(x)}$$
where $b$ is a string of length $m$, and $\oplus$ represents the sum modulus $2^m$. Consider the following protocol:
<ul>
  <li>Prepare the state 
    $$\frac{1}{\sqrt{N}} \sum_{x\in Z_N} \ket{x}$$
    These first qubits constitute the <b>first register</b>.
  </li>
  <li> Attach a <b>second register</b> register in stae $\ket{0^m}$.</li>
  <li> Query the oracle $O_f$ on the current input.</li>
  <li> Measure the second register in the computational basis.</li>
  <li> Apply the Quantum Fourier Transform to the first register.</li> 
  <li> Measure the first register. </li>
</ul>

<ol>
  <li>Let $c \in C$ be fixed. Show that the probability of getting $c$ as outcome for the measurement on the second register is $\frac{1}{s}$.</li>
  <li>Write the state of the first register after having obtained $c$ as measurement outcome on the second register.</li>
</ol>

<b>Hint.</b> It could be useful to define the auxiliary function 
$$f_c(x) = \begin{cases}1 & \mbox{ if } f(x)=c \\ 0 & \mbox { otherwise}\end{cases}$$

## 3.3.c The Fourier coefficients /1 (1 pt.)
Consider a function $g: \mathbb{Z}_N \to C$ and a number $t \in \mathbb{Z}_N$. Let $k: \mathbb{Z}_N \to C$ be another function such that $k(x) = g(x+t)$. Show that $g$ and $k$ have the same Fourier coefficients, except for a multiplicative factor of absolute value 1.

Notice that from this it follows that the measurement outcome on the first register does not depend on the $c$ we measured on the second register. Without loss of generality, we can therefore assume to have measured $f(0)=c$. Therefore:
$$f_c(x)=\begin{cases}1 & \mbox{ if } x=0,s,2s,\ldots \\ 0 & \mbox{ otherwise}\end{cases}$$

## 3.3.d The Fourier coefficients /2 (1 pt.)
Prove that $f_c(x)$ defined in this way has Fourier coefficients given by:
$$\hat{f_c}(\gamma) = \begin{cases}\frac{1}{s} & \mbox{ if } \gamma \in \{0, N/s, 2N/s,\ldots\} \\ 0 & \mbox{ otherwise}\end{cases}$$

It follows that, when performing the measurement on the first register, we are sampling uniformly a value of $\gamma$ from the set $\{0, N/s, 2N/s, \ldots \}$, or equivalently from the $\gamma$ values such that $\gamma s =0 \mod N$.

## 3.3.e Finding the period (1 pt.)
Describe how the measurement outcomes obtained from the measurement on the first register can be exploited to find the period of the function $f$.

# 3.4 - Shor's factoring (5 pts. total)
The aim of this exercise is to build a specific implementation of Shor's factoring algorithm, which will allow factorisation of the number $15$. 

## 3.4.a Classical auxiliary functions  (1 pt.)
First of all, some classical subroutines are required. Write the following Python functions:
<ol>
    <li>a <code>gcd(x,y)</code> function, which computes the greatest common divisor between the numbers <code>x</code> and <code>y</code>. </li>
    <li>a <code>verify_factor(N,x)</code> function, which verifies whether <code>x</code> is a factor of <code>N</code>. 
</ol>
Discuss the scaling of the computational complexity of the latter function, depending on the number of bits required to encode $N$, in terms of the $NP$ class.

In [ ]:
def gcd(x, y):
    while y != 0:
        (x, y) = (y, x % y)
        return x

def verify_factor(N, x):
    if (N%x == 0) and (N >= x) :
         return True
    else:
        return False

Latter function scales exponentially


## 3.4.b Classical period finding (1 pt.)
Consider the function
$$f(x) = a^x \mod N$$
where $a$, $x$ and $N$ are numbers in $\mathbb{N}$. It can be shown that the function $f(x)$ is periodic, and that each value of $f(x)$ appears only once in each period.

Write a Python function <code>classical_period_find(a,N)</code> able to compute the period of the function $f(x)$ for given $a$ and $N$. Discuss the complexity of the function, in terms of the bits required to encode $N$.

Write also a Python function <code>period_verify(a,N,p)</code>, which verifies whether the proposed $p$ is the right period for $f(x)$. Discuss its complexity in terms of the bits required to encode $N$. Discuss the relation between <code>classical_period_find</code> and <code>period_verify</code> in terms of the $NP$ complexity class.

In [ ]:
def period_verify(a, N, ans):
    if classical_period_find(a, N) == ans:
        return True
    else:
        return False

def classical_period_find(a, N):
    f = lambda x, a, N: a**x % N 
    f_initial= f(1, a, N) 
    x = 2
    while f_initial != f(x, a, N): 
        x+=1 
        
    return x -2 +1 



classical_period_find(2, 15)



4

In [ ]:
period_verify(2, 15,4)

True

Number of bits needed to encode N sclaes logarithmically. Both functions are in the NP class

## 3.4.c Quantum period finding (1 pt.)
For the specific case of $a=2$, $N=15$, write the Qiskit code for quantum period finding (Shor's algorithm). 

In [ ]:
n_count = 4
def U15(n, p): 
    if n not in [2,7,8,11,13]:
        return 0
    
    Q = QuantumCircuit(4)
    
    for i in range(p):
        if n in [2,13]:
            Q.swap(0,1)
            Q.swap(1,2)
            Q.swap(2,3)
        if n in [7,8]:
            Q.swap(2,3)
            Q.swap(1,2)
            Q.swap(0,1)
        if n == 11:
            Q.swap(1,3)
            Q.swap(0,2)
        if n in [7,11,13]:
            for q in range(4):
                Q.x(q)
                
    Q = Q.to_gate()
    c_Q = Q.control()
    return c_Q


def create_shor_qc(a, N, n_count = 4):
    qc = QuantumCircuit(n_count + 4, n_count)

    
    qc.h(range(n_count))
        
    
    qc.x(3 + n_count)

   
    for q in range(n_count):
        qc.append(U15(a, 2**q), 
                [q] + [i+n_count for i in range(4)])

    
    inv_qft(qc,n_count)
    return qc

a=2
N=15

qc = create_shor_qc(a, N)


qc.measure(range(n_count), range(n_count))
qc.draw()   

┌───┐                                                                    »
q_0: ┤ H ├────────■───────────────────────────────────────────────────────────»
     ├───┤        │                                                           »
q_1: ┤ H ├────────┼────────────────■─────────────────────────────────X────────»
     ├───┤        │                │                                 │        »
q_2: ┤ H ├────────┼────────────────┼────────────────■────────────────X────────»
     ├───┤        │                │                │                         »
q_3: ┤ H ├────────┼────────────────┼────────────────┼────────────────■────────»
     └───┘┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐┌───────┴───────┐»
q_4: ─────┤0              ├┤0              ├┤0              ├┤0              ├»
          │               ││               ││               ││               │»
q_5: ─────┤1              ├┤1              ├┤1              ├┤1              ├»
          │  circuit-8237 ││  circuit-8247 ││  circuit-8260 ││  circuit-8279 │»
q_6: ─────┤2              ├┤2              ├┤2              ├┤2              ├»
     ┌───┐│               ││               ││               ││               │»
q_7: ┤ X ├┤3              ├┤3              ├┤3              ├┤3              ├»
     └───┘└───────────────┘└───────────────┘└───────────────┘└───────────────┘»
c: 4/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«        ┌───┐                                                            ┌─┐»
«q_0: ─X─┤ H ├─■──────────────■───────────────────■───────────────────────┤M├»
«      │ └───┘ │P(-π/2) ┌───┐ │                   │                       └╥┘»
«q_1: ─┼───────■────────┤ H ├─┼─────────■─────────┼──────────────■─────────╫─»
«      │                └───┘ │P(-π/4)  │P(-π/2)  │        ┌───┐ │         ║ »
«q_2: ─┼──────────────────────■─────────■─────────┼────────┤ H ├─┼─────────╫─»
«      │                                          │P(-π/8) └───┘ │P(-π/4)  ║ »
«q_3: ─X──────────────────────────────────────────■──────────────■─────────╫─»
«                                                                          ║ »
«q_4: ─────────────────────────────────────────────────────────────────────╫─»
«                                                                          ║ »
«q_5: ─────────────────────────────────────────────────────────────────────╫─»
«                                                                          ║ »
«q_6: ─────────────────────────────────────────────────────────────────────╫─»
«                                                                          ║ »
«q_7: ─────────────────────────────────────────────────────────────────────╫─»
«                                                                          ║ »
«c: 4/═════════════════════════════════════════════════════════════════════╩═»
«                                                                          0 »
«                             
«q_0: ────────────────────────
«               ┌─┐           
«q_1: ──────────┤M├───────────
«               └╥┘     ┌─┐   
«q_2: ─■─────────╫──────┤M├───
«      │P(-π/2)  ║ ┌───┐└╥┘┌─┐
«q_3: ─■─────────╫─┤ H ├─╫─┤M├
«                ║ └───┘ ║ └╥┘
«q_4: ───────────╫───────╫──╫─
«                ║       ║  ║ 
«q_5: ───────────╫───────╫──╫─
«                ║       ║  ║ 
«q_6: ───────────╫───────╫──╫─
«                ║       ║  ║ 
«q_7: ───────────╫───────╫──╫─
«                ║       ║  ║ 
«c: 4/═══════════╩═══════╩══╩═
«                1       2  3

In [ ]:
result = list(aer_sim.run(transpile(qc, aer_sim)).result().get_counts().keys())
end_result = [int(r, 2) for r in result]
end_result

[8, 0, 4, 12]

Note: Circuit seems to be nearly doing the right thing but the order is mixed for some reason

Pass the most frequent outcomes to the following function, which estimates the compatible periods using a variant of the continuous-fractions algorithm. You can verify which one of the candidates is the "right" period using the <code>period_verify</code> function.

In [ ]:
def period_guess(x):
    Range = 16
    
    if x < Range/2:
        x = Range - x
        
    
    
    g = []
    Real = x / Range
    b= 1.
    zero = 0
    one = 1
    two = 0
    
 
    
    for denominator in range(1, x):
            numerator = round(denominator * Real)
            estimated = numerator / denominator 
            error = abs(estimated - Real)
        
            zero = one
            one = two
            two = error
        
            if one  <=  b and one < zero and one < two:
                repeat_period = denominator - 1
                g.append(denominator - 1)
                b = one
            if 0 in g:
                g.remove(0)
        
    return g

def period(result): 

    for r in result:
        guess = period_guess(r)
        
        for g in guess:
            if period_verify(a, N, g):
                return g
            

p = period(end_result)

print(period_verify(a, N, p))


True


## 3.4.d Prime factoring (1 pt.)
Now, exploit the <code>gcd</code> function to return the factorization of the number $15$ from the Shor's algorithm. If $p$ is the period of the function $f(x)$, then one of $N$'s prime factor might be given by 
$$\gcd\left(N, a^{p/2}+1\right)$$
or by
$$\gcd\left(N, a^{p/2}-1\right)$$

## 3.4.e Order finding and period finding (1 pt.)
In this exercise, you have built Shor's factorization algorithm in a sligthly different way than in the class, based on the <b>period finding</b> rather than on the <b>order finding</b>. It is anyhow easy to show that order finding is a subclass of period finding.

Period finding, as discussed above, aims at finding the least positive $r$ such that:
$$f(x+r) = f(x) \,\,\,\,\, \forall x$$

Order finding considers a specific function of the form:
$$ f(x) = a^x \mod N$$
and aims at finding the least $x\neq 0$ such that $f(x) = 1$. 

Show that order finding is a subclass of period finding. 

# 3.5 - A Variational Quantum Eigensolver (5 pts.)
Variational Quantum Eigensolvers are hybrid algorithms, putting together elements of classical and quantum computation. They can be used to compute the ground state energy of a system. In this case, we will consider a system of two qubits, having Hamiltonian $\mathcal{H}$ given by:
$$\mathcal{H} = \ket{00}\bra{00} - \ket{10}\bra{10} - \ket{01}\bra{01} + \ket{11}\bra{11}$$

## 3.5.a Decomposition in terms of Pauli operators (1 pt.)
The first step towards a VQE is writing $\mathcal{H}$ as a linear combination of the Pauli operators $\sigma_x$, $\sigma_y$ and $\sigma_z$, plus the identity (which could be called for instance $\sigma_0$, and will be assumed to be a part of the Pauli operators set in the following). This is always possible, since they constitute a basis for the space of $2\times 2$ complex matrices. Therefore, the tensor products of Pauli matrices will constitute a basis for the space of $4\times 4$ complex matrices.

Write a Python script <code>decompose_in_Pauli_matrices</code> which, given as input a $2\times 2$ matrix, decomposes it on the Pauli matrices basis, giving as output the coefficients of the linear combination. Apply to the specific case of the Hamiltonian $\mathcal{H}$.

In [ ]:
def decompose_in_Pauli_matrices(H):
    Ham = H[0,0]*np.outer(np.kron(zero,zero),np.kron(zero,zero).conj().T)+H[1,0]*np.outer(np.kron(one,zero),np.kron(one,zero).conj().T)+H[0,1]*np.outer(np.kron(zero,one),np.kron(zero,one).conj().T)+H[1,1]*np.outer(np.kron(one,one),np.kron(one,one).conj().T)
  
    coeffs = np.zeros((4,4))
    for i in range(4):
        for j in range(4):
            coeffs[i,j] = (np.trace(np.kron(S[i],S[j])@Ham))/4
    
        
    
    return coeffs.flatten()
    
H = np.array([[1,-1],[-1,1]])

print(decompose_in_Pauli_matrices(H))


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


C:\Users\Pa\AppData\Local\Temp/ipykernel_17528/2919520292.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs[i,j] = (np.trace(np.kron(S[i],S[j])@Ham))/4


## 3.5.b Measurement preparation (1 pt.)
In the previous step, we have written $\mathcal{H}$ as a linear combination of Pauli matrices; that is:
$$\mathcal{H} = \sum_{\alpha\beta} h_{\alpha\beta} \sigma_\alpha \otimes \sigma_\beta$$
where $\alpha$ and $\beta$ take values in the set $\{0,x,y,z\}$. Hence, the expectation value of the Hamiltonian on a state $\ket{\psi}$ is given by:
$$\langle H \rangle_\psi = \sum_{\alpha\beta} h_{\alpha\beta} \langle\sigma_\alpha \otimes \sigma_\beta\rangle_\psi$$

The usual Qiskit measurement only allows us to measure on the computational basis, i.e. $\sigma_z^{\otimes n}$. However, we would like to be able to measure any couple of tensorized Pauli operators $\sigma_\alpha \otimes \sigma_\beta$. We need therefore to use unitary operators to transform a measurement of $\sigma_\alpha\otimes\sigma_\beta$ couple into a measurement of a single qubit, on $\sigma_z$.

As an example, we propose a circuit to measure $\sigma_z\otimes\sigma_z$, which is given by:

In [ ]:
measure_z_z = qiskit.QuantumCircuit(2,1)
measure_z_z.cx(1,0)
measure_z_z.measure(0,0)

measure_z_z.draw()

┌───┐┌─┐
q_0: ┤ X ├┤M├
     └─┬─┘└╥┘
q_1: ──■───╫─
           ║ 
c: 1/══════╩═
           0

Create in Qiskit suitable circuits to do this for each one of the couples appearing in the Hamiltonian $\mathcal{H}$. 

In [ ]:
measure_i_x = qiskit.QuantumCircuit(2,1)
measure_i_x.swap(0,1)
measure_i_x.h(0)
measure_i_x.measure(0,0)

measure_i_x.draw()                          

┌───┐┌─┐
q_0: ─X─┤ H ├┤M├
      │ └───┘└╥┘
q_1: ─X───────╫─
              ║ 
c: 1/═════════╩═
              0

In [ ]:
measure_i_y = qiskit.QuantumCircuit(2,1)
measure_i_y.swap(0,1)
measure_i_y.h(0)
measure_i_y.sdg(0)
measure_i_y.measure(0,0)

measure_i_y.draw()   

┌───┐┌─────┐┌─┐
q_0: ─X─┤ H ├┤ Sdg ├┤M├
      │ └───┘└─────┘└╥┘
q_1: ─X──────────────╫─
                     ║ 
c: 1/════════════════╩═
                     0

In [ ]:
measure_i_z = qiskit.QuantumCircuit(2,1)
measure_i_z.swap(0,1)
measure_i_z.measure(0,0)

measure_i_z.draw()

┌─┐
q_0: ─X─┤M├
      │ └╥┘
q_1: ─X──╫─
         ║ 
c: 1/════╩═
         0

In [ ]:
measure_x_i = qiskit.QuantumCircuit(2,1)
measure_x_i.h(0)
measure_x_i.measure(0,0)

measure_x_i.draw()

┌───┐┌─┐
q_0: ┤ H ├┤M├
     └───┘└╥┘
q_1: ──────╫─
           ║ 
c: 1/══════╩═
           0

In [ ]:
measure_x_x = qiskit.QuantumCircuit(2,1)
measure_x_x.h(0)
measure_x_x.h(1)
measure_x_x.cx(1,0)
measure_x_x.measure(0,0)

measure_x_x.draw()

┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ X ├┤M├
     ├───┤└─┬─┘└╥┘
q_1: ┤ H ├──■───╫─
     └───┘      ║ 
c: 1/═══════════╩═
                0

In [ ]:
measure_x_y = qiskit.QuantumCircuit(2,1)
measure_x_y.h(0)
measure_x_y.sdg(1)
measure_x_y.h(1)
measure_x_y.cx(1,0)
measure_x_y.measure(0,0)

measure_x_y.draw()

┌───┐      ┌───┐┌─┐
q_0: ─┤ H ├──────┤ X ├┤M├
     ┌┴───┴┐┌───┐└─┬─┘└╥┘
q_1: ┤ Sdg ├┤ H ├──■───╫─
     └─────┘└───┘      ║ 
c: 1/══════════════════╩═
                       0

In [ ]:
measure_x_z = qiskit.QuantumCircuit(2,1)
measure_x_z.h(0)
measure_x_z.cx(1,0)
measure_x_z.measure(0,0)

measure_x_z.draw()

┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ X ├┤M├
     └───┘└─┬─┘└╥┘
q_1: ───────■───╫─
                ║ 
c: 1/═══════════╩═
                0

In [ ]:
measure_y_i = qiskit.QuantumCircuit(2,1)
measure_y_i.sdg(0)
measure_y_i.h(0)
measure_y_i.measure(0,0)

measure_y_i.draw()

┌─────┐┌───┐┌─┐
q_0: ┤ Sdg ├┤ H ├┤M├
     └─────┘└───┘└╥┘
q_1: ─────────────╫─
                  ║ 
c: 1/═════════════╩═
                  0

In [ ]:
measure_y_x = qiskit.QuantumCircuit(2,1)
measure_y_x.sdg(1)
measure_y_x.h(1)
measure_y_x.h(0)
measure_y_x.cx(1,0)
measure_y_x.measure(0,0)

measure_y_x.draw()

┌───┐      ┌───┐┌─┐
q_0: ─┤ H ├──────┤ X ├┤M├
     ┌┴───┴┐┌───┐└─┬─┘└╥┘
q_1: ┤ Sdg ├┤ H ├──■───╫─
     └─────┘└───┘      ║ 
c: 1/══════════════════╩═
                       0

In [ ]:
measure_y_y = qiskit.QuantumCircuit(2,1)
measure_y_y.sdg(1)
measure_y_y.h(1)
measure_y_y.sdg(0)
measure_y_y.h(0)
measure_y_y.cx(1,0)
measure_y_y.measure(0,0)

measure_y_y.draw()

┌─────┐┌───┐┌───┐┌─┐
q_0: ┤ Sdg ├┤ H ├┤ X ├┤M├
     ├─────┤├───┤└─┬─┘└╥┘
q_1: ┤ Sdg ├┤ H ├──■───╫─
     └─────┘└───┘      ║ 
c: 1/══════════════════╩═
                       0

In [ ]:
measure_y_z = qiskit.QuantumCircuit(2,1)
measure_y_z.sdg(0)
measure_y_z.h(0)
measure_y_z.cx(1,0)
measure_y_z.measure(0,0)

measure_y_z.draw()

┌─────┐┌───┐┌───┐┌─┐
q_0: ┤ Sdg ├┤ H ├┤ X ├┤M├
     └─────┘└───┘└─┬─┘└╥┘
q_1: ──────────────■───╫─
                       ║ 
c: 1/══════════════════╩═
                       0

In [ ]:
measure_z_i = qiskit.QuantumCircuit(2,1)
measure_z_i.measure(0,0)

measure_z_i.draw()

┌─┐
q_0: ┤M├
     └╥┘
q_1: ─╫─
      ║ 
c: 1/═╩═
      0

In [ ]:
measure_z_x = qiskit.QuantumCircuit(2,1)
measure_z_x.h(1)
measure_z_x.cx(1,0)
measure_z_x.measure(0,0)

measure_z_x.draw()

┌───┐┌─┐
q_0: ─────┤ X ├┤M├
     ┌───┐└─┬─┘└╥┘
q_1: ┤ H ├──■───╫─
     └───┘      ║ 
c: 1/═══════════╩═
                0

In [ ]:
measure_z_y = qiskit.QuantumCircuit(2,1)
measure_z_y.sdg(1)
measure_z_y.h(1)
measure_z_y.cx(1,0)
measure_z_y.measure(0,0)

measure_z_y.draw()

┌───┐┌─┐
q_0: ────────────┤ X ├┤M├
     ┌─────┐┌───┐└─┬─┘└╥┘
q_1: ┤ Sdg ├┤ H ├──■───╫─
     └─────┘└───┘      ║ 
c: 1/══════════════════╩═
                       0

## 3.5.c Ansatz (1 pt.)
We will need to somehow define a <b>family</b> of states in which we would like to minimize the expectation value of the energy. To do so, we consider an <b>ansatz</b>, expressing our state as a function of some parameters. In this case, we will consider the ansatz:
$$\ket{\psi(\theta)} = \left[\left(RX(\theta)\otimes\mathbb{I}\right) CNOT(H\otimes\mathbb{I})\right]\ket{00}$$
Create a Python function which generates a quantum circuit having initial state $\ket{00}$, and transforming it into $\ket{\psi(\theta)}$. It could be useful to look at the documentation of the <code>RX</code> gate <a href="https://qiskit.org/documentation/stubs/qiskit.circuit.library.RXGate.html">here</a>.

In [ ]:
Anzats = qiskit.QuantumCircuit(2,1)

theta = Parameter('θ')

Anzats.h(0)
Anzats.cx(0,1)
Anzats.rx(theta,0)

Anzats.draw()



┌───┐     ┌───────┐
q_0: ┤ H ├──■──┤ Rx(θ) ├
     └───┘┌─┴─┐└───────┘
q_1: ─────┤ X ├─────────
          └───┘         
c: 1/═══════════════════

## 3.5.d Measuring energy (1 pt.)
With the building blocks created in the previous steps, we are now able to compute the expectation value
$$\langle \mathcal{H} \rangle_{\psi(\theta)} = \bra{\psi(\theta)}\mathcal{H}\ket{\psi(\theta)}$$
Create a Python function <code>vqe_step</code> taking as input the value of $\theta$, which performs a custom-defined <code>number_runs</code> times all the Pauli measurements necessary to compute the expectation value of the energy. The function should return the estimated expectation value of the energy.

In [ ]:
numbers = decompose_in_Pauli_matrices(H)

#print(numbers)

sim = Aer.get_backend('aer_simulator')

number_runs = 2**17 # number of samples used for statistics

A = 1.47e-6 #unit of A is eV

def vqe_step(theta):

    Anzats = qiskit.QuantumCircuit(2,1)

    
    
    Anzats.h(0)
    Anzats.cx(0,1)
    Anzats.rx(theta,0)

   


    

    
                
    E_sim = []
    for state_init in [Anzats]:
        Energy_meas = []
        for measure_circuit in [measure_z_z]:
    
            # run the circuit with the selected measurement and get the number of samples that output each bit value
            qc = state_init.compose(measure_circuit)
            qc_trans = transpile(qc, sim)
            counts = sim.run(qc_trans, shots=number_runs).result().get_counts()
          

            # calculate the probabilities for each computational basis
            probs = {}
            for output in ['0','1']:
                if output in counts:
                    probs[output] = counts[output]/number_runs
                else:
                    probs[output] = 0
            
            Energy_meas.append(  probs['0'] - probs['1'] )
 
        E_sim.append(np.sum(np.array(Energy_meas)))
    
    return(E_sim[0])


    


C:\Users\Pa\AppData\Local\Temp/ipykernel_17528/2919520292.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs[i,j] = (np.trace(np.kron(S[i],S[j])@Ham))/4


## 3.5.e Classical minimization (1 pt.)
The function <code>vqe_step</code> can now be used as part of a classical optimization loop. 

Write Python code to find the value of $\theta$ (let's call it $\tilde{\theta}$) corresponding to the lowest expectation value of the energy, as computed by the <code>vqe_step</code> function. The state $\ket{\psi(\tilde{\theta})}$ is therefore our best estimation for the ground state of the Hamiltonian $\mathcal{H}$. It could be useful to exploit the highly-optimized minimization algorithms available in <code>scipy</code>, have a look <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html">here</a>. 

Output the value of $\theta$ yielding the lowest energy, and the corresponding state $\ket{\psi(\tilde{\theta})}$. Diagonalize the Hamiltonian $\mathcal{H}$ (by hand or with Python) and discuss whether the state $\ket{\psi(\tilde{\theta})}$ found is the actual ground state of the Hamiltonian.

In [ ]:
scipy.optimize.minimize_scalar(vqe_step)



     fun: -1.0
    nfev: 29
     nit: 24
 success: True
       x: 3.1483037753879253

In [ ]:
print('Our optimal value of theta is pi')

Our optimal value of theta is pi


In [ ]:
qc = qiskit.QuantumCircuit(2)

theta = np.pi

qc.h(0)
qc.cx(0,1)
qc.rx(theta,0)

qc.draw()



┌───┐     ┌───────┐
q_0: ┤ H ├──■──┤ Rx(π) ├
     └───┘┌─┴─┐└───────┘
q_1: ─────┤ X ├─────────
          └───┘

In [ ]:

# Apply initialisation operation to the 0th qubit
qc.save_statevector()   # Tell simulator to save statevector
qobj = assemble(qc)     # Create a Qobj from the circuit for the simulator to run
result = sim.run(qobj).result() # Do the simulation and return the result

out_state = result.get_statevector()
print(out_state) # Display the output state vector




Statevector([4.32978028e-17+0.j        , 0.00000000e+00-0.70710678j,
             0.00000000e+00-0.70710678j, 4.32978028e-17+0.j        ],
            dims=(2, 2))


In [ ]:
Ham = np.diag([1,-1,-1,1])
#print(Ham)

eigs = np.linalg.eig(Ham)
print(eigs[0])
print(eigs[1])

print('Our real Ground state wavefns are (0,1,0,0) and (0,0,1,0)')

[ 1. -1. -1.  1.]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Our real Ground state wavefns are (0,1,0,0) and (0,0,1,0)


In [ ]:
print('The ground state that we get from our VQE analysis does not correspond exactly to the ')
print('real ground state of the Hamiltonian, which are two degenerate energy states.')
print('However, the ground state we have found is a linear combination of these two states')
print('With a normalisation factor which I consider to be a reasonable approximation of ')
print('the ground state')

The ground state that we get from our VQE analysis does not correspond exactly to the 
real ground state of the Hamiltonian, which are two degenerate energy states.
However, the ground state we have found is a linear combination of these two states
With a normalisation factor which I consider to be a reasonable approximation of 
the ground state


$|\psi(\tilde{\theta})\rangle=\frac{1}{\sqrt{2}}(0,-i,-i, 0)$